In [4]:
from pyspark import SparkContext,SparkConf
import numpy as np
#import ujson
import pickle
import time

In [5]:
def safe_parse(raw_json):
    try:
        json_object = ujson.loads(raw_json)
        if 'user' not in json_object.keys():
            return False
        pass
    except ValueError, e:
        return False
    return True

def create_tuple(raw_json):
    json_object = ujson.loads(raw_json)
    return (json_object['user']['id_str'],json_object['text'])

In [6]:
partition = pickle.load(open("users-partition.pickle","r"))

def create_group_tuple(x):
    try: 
        gid=partition[x]
        return (gid,1)
    except:
        return (7,1)

In [7]:
conf=SparkConf()
conf.setMaster('local[2]')
conf.setAppName('tuning-test')
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
sc = SparkContext(conf=conf)

Exception: Java gateway process exited before sending the driver its port number

In [5]:
print "Kryo Serializer"

DataRDD=sc.textFile(','.join([line[:-1] for line in open('hw2-files.txt')]))

st=time.time()

UserTextRDD = DataRDD.map(lambda x: x.encode('utf-8')).filter(lambda x: safe_parse(x))\
                .map(lambda x: create_tuple(x))
GroupCountRDD=UserTextRDD.map(lambda x: create_group_tuple(x[0])).reduceByKey(lambda x,y: x+y).sortByKey().cache()

print time.time()-st

Kryo Serializer
8.29482221603


In [6]:
sc.stop()

In [7]:
sc = SparkContext('local[2]', 'tuning-test2')

In [8]:
print "Java Serializer"

DataRDD=sc.textFile(','.join([line[:-1] for line in open('./Data/hw2-files.txt')]))

st=time.time()

UserTextRDD = DataRDD.map(lambda x: x.encode('utf-8')).filter(lambda x: safe_parse(x))\
                .map(lambda x: create_tuple(x))
GroupCountRDD=UserTextRDD.map(lambda x: create_group_tuple(x[0])).reduceByKey(lambda x,y: x+y).sortByKey().cache()

print time.time()-st

Java Serializer
7.79762101173


In [9]:
sc.stop()